# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выберите данные по странам своей группы (совместно): <br>
    3530203_80101: Spain, Portugal, France, Italy, Belgium<br>
    3530203_80102: Germany, Netherlands <br>
    3530903_80301: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_80302: Sweden, Finland, Norway, Denmark, Iceland<br>
    
2. Попытайтесь найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Найдите и объясните хотя бы 5 правил.
3. Выведите эти правила в отдельных ячейках. 
4. Подумайте, как можно было бы использовать полученные правила на практике.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules

In [2]:
data = pd.read_csv('lastfm.csv')
data = data[data.country.isin(['Spain', 'Portugal', 'France', 'Italy', 'Belgium'])]
data['country'].unique()

array(['Portugal', 'Italy', 'Spain', 'France', 'Belgium'], dtype=object)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27214 entries, 83 to 289762
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     27214 non-null  int64 
 1   artist   27214 non-null  object
 2   sex      27214 non-null  object
 3   country  27214 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [4]:
data['country'].value_counts()

Spain       9322
France      5962
Italy       5717
Belgium     3331
Portugal    2882
Name: country, dtype: int64

In [5]:
#First, what you need to accomplish is to transform the data given here into an incidence matrix, where each listener represents a row, with 0s and 1s across the columns. This indicates if he or she has played a certain artist or not. Then, the support for each of the listed 1004 artists needs to be calculated by displaying the support for all artists with support larger than 0.08

#Then, construct the association rules using the function Apriori in the R package arules and then look for artists (or groups of artists) who have support that is larger than 0.01 (1%). After the calculation is checked, another music collection of an artist turns out to be larger than 0.50 (50%). - KAGGLE

# Строю матрицу инцидентности по артистам. IM = incidence matrix
data_artist_grouped = data.groupby(['user', 'sex', 'country'])['artist'].apply(','.join).reset_index()
IM_artists = data_artist_grouped['artist'].str.get_dummies(',')

IM_artists.head() # Матрица инцидентности по артистам

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Правило 1: Кому следует и кому не следует советовать AC/DC? (Apriori)


In [6]:
# Пояснение к метрикам в результате "что есть что"
# https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/
apriori_frequent_sets = apriori(IM_artists, min_support=0.01, use_colnames=True)  # минимальная поддержка из KAGGLE
apriori_rules = association_rules(apriori_frequent_sets, metric="confidence", min_threshold=0.2)
apriori_rules[{'antecedents', 'consequents', 'lift', 'conviction'}]

,consequents,antecedents,lift,conviction
0,(30 seconds to mars),(fall out boy),9.214286,1.334302
1,(fall out boy),(30 seconds to mars),9.214286,1.495263
2,(linkin park),(30 seconds to mars),5.962185,1.832276
3,(muse),(30 seconds to mars),2.462117,1.445384
4,(mogwai),(65daysofstatic),10.413405,2.043042
...,...,...,...,...
8308,"(interpol, the strokes, muse)","(radiohead, the killers)",11.315789,1.325581
8309,"(the strokes, muse, the killers)","(interpol, radiohead)",9.356044,1.267935
8310,"(interpol, the strokes, radiohead)","(muse, the killers)",9.808756,1.244923
8311,"(the strokes, radiohead, the killers)","(interpol, muse)",13.591954,1.323172


Чем больше lift, тем больше вероятность того, что предпосылка и следствие будут встречены вместе. То есть тот, кто слушал одну группу будет слушать и другую. Если lift = 1, значит предпосылка и следствие независимы
Чем больше conviction, тем больше "убежденность" в том, что предпосылка и следствие связаны. Если предпосылка и следствие всегда вместе, то conviction будет равен бесконечности

In [7]:
rule = apriori_rules[(apriori_rules['antecedents'] == {'ac/dc'})].sort_values(by=['lift', 'conviction'], ascending=False)
print('Какие группы вероятно подойдут любителям AC/DC? Спойлер - "рок"')
rule[{'antecedents', 'consequents', 'lift', 'conviction'}]

Какие группы вероятно подойдут любителям AC/DC? Спойлер - "рок"


,consequents,antecedents,lift,conviction
35,(motörhead),(ac/dc),6.606984,1.224942
21,(black sabbath),(ac/dc),4.504762,1.206220
27,(iron maiden),(ac/dc),4.028297,1.314925
32,(metallica),(ac/dc),3.243429,1.461123
30,(led zeppelin),(ac/dc),3.185510,1.314453
37,(queen),(ac/dc),2.527062,1.169494
36,(pink floyd),(ac/dc),2.045405,1.185854
40,(the beatles),(ac/dc),1.586698,1.141093
38,(radiohead),(ac/dc),0.882238,0.964620


При этом, если lift < 1, то предпосылка и следствие скорее имеют обратную зависимость. Иначе говоря - не надо давать такие группы слушать

In [8]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}][rule['lift'] < 1]

,consequents,antecedents,lift,conviction
38,(radiohead),(ac/dc),0.882238,0.96462


### Правило 2: Можно ли судить о предпочтениях по комбинации групп? (FPGrowth)

In [9]:
fpgrowth_frequent_sets = fpgrowth(IM_artists, min_support=0.01, use_colnames=True).sort_values(by='support', ascending=False)
rule  = association_rules(fpgrowth_frequent_sets, metric="confidence", min_threshold=0.1)
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)

,consequents,antecedents,lift,conviction
9301,(gamma ray),(helloween),23.650000,1.684083
9302,(helloween),(gamma ray),23.650000,2.436575
9046,(helloween),(iced earth),22.523810,2.274137
9047,(iced earth),(helloween),22.523810,1.764482
5841,(children of bodom),(arch enemy),22.336111,2.249146
...,...,...,...,...
6500,(coldplay),(ac/dc),0.860460,0.968672
467,(radiohead),(metallica),0.842136,0.953136
466,(metallica),(radiohead),0.842136,0.978275
1750,(radiohead),(queen),0.821597,0.947359


Объяснение правила аналогичное предыдущему, но что интересно - с алгоритмом FPGrowth высокий показатель lift у комбинации групп

### ОФФТОП

In [10]:
#Собираю дополнительные таблицы так, потому что не могу придумать, как иначе
user_sex = data[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
user_country = data[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
user_artist = data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
user_sex_country = pd.concat([user_sex, user_country], axis=1)
user_sex_artist = pd.concat([user_sex, user_artist], axis=1)
user_country_artist = pd.concat([user_country, user_artist], axis=1)
user_all = pd.concat([user_sex, user_country, user_artist], axis=1)

### Правило 3: Есть ли зависимость от пола?

In [11]:
sex_artist_frequent_sets = apriori(user_sex_artist, min_support=0.01, use_colnames=True).sort_values(by='support', ascending=False)
rule = association_rules(sex_artist_frequent_sets, metric="confidence", min_threshold=0.1)

In [12]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'m'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/939752960.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'m'}]


,consequents,antecedents,lift,conviction
10,(metallica),(m),1.135346,1.019409
14,(daft punk),(m),1.122718,1.014165
8,(pink floyd),(m),1.101692,1.015482
20,(led zeppelin),(m),1.080410,1.008880
13,(nirvana),(m),1.079065,1.011396
16,(red hot chili peppers),(m),1.046402,1.005391
0,(radiohead),(m),1.042210,1.013322
4,(the beatles),(m),1.011981,1.002531
2,(coldplay),(m),0.974585,0.994144
22,(the killers),(m),0.961382,0.995253


In [13]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'f'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/2082415823.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'f'}]


,consequents,antecedents,lift,conviction
1135,(death cab for cutie),(f),1.607906,1.047771
580,(amy winehouse),(f),1.527870,1.055228
504,(placebo),(f),1.461882,1.053250
485,(the cure),(f),1.392268,1.047486
1137,(david bowie),(f),1.273462,1.027133
683,(arctic monkeys),(f),1.263355,1.030655
880,(depeche mode),(f),1.261510,1.028750
1564,(oasis),(f),1.233377,1.021625
134,(muse),(f),1.178449,1.039078
813,(the killers),(f),1.137019,1.017215


При попытке отследить женщин и мужчин, как причину выбора музыкальной группы было выяснено, что половая принадлежность особо не влияет на вкус в музыке. Наиболее вероятно из этого исследования можно сказать, что женщинам в среднем нравится "The Cure" - знать ЦА полезно в маркетинге

### Правило 4: есть ли предпочтения по странам?

In [14]:
country_artist_frequent_sets = apriori(user_country_artist, min_support=0.01, use_colnames=True).sort_values(by='support', ascending=False)
rule = association_rules(country_artist_frequent_sets, metric="confidence", min_threshold=0.1)

In [15]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Spain'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/753675912.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Spain'}]


,consequents,antecedents,lift,conviction
18,(the killers),(Spain),1.528010,1.069767
96,(amy winehouse),(Spain),1.226902,1.023015
76,(franz ferdinand),(Spain),1.216592,1.023499
148,(queen),(Spain),1.162778,1.015691
10,(muse),(Spain),1.089949,1.019323
4,(coldplay),(Spain),1.060821,1.014298
121,(arctic monkeys),(Spain),1.051630,1.005865
22,(the beatles),(Spain),0.896937,0.978741
143,(nirvana),(Spain),0.839720,0.977668
12,(radiohead),(Spain),0.740614,0.925250


In [16]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Portugal'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/3759632400.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Portugal'}]


,consequents,antecedents,lift,conviction
9281,(beirut),(Portugal),2.269492,1.084434
6681,(arcade fire),(Portugal),2.149234,1.086573
6643,(pearl jam),(Portugal),2.136333,1.092061
1761,(nirvana),(Portugal),1.708530,1.112315
6683,(bloc party),(Portugal),1.675674,1.065284
1895,(metallica),(Portugal),1.661593,1.102621
4978,(arctic monkeys),(Portugal),1.534665,1.064267
13437,(linkin park),(Portugal),1.466111,1.044569
13421,(sigur rós),(Portugal),1.418433,1.041355
6658,(the killers),(Portugal),1.342055,1.044113


In [17]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'France'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/4075796453.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'France'}]


,consequents,antecedents,lift,conviction
650,(gorillaz),(France),2.078046,1.060200
152,(daft punk),(France),1.526289,1.063716
224,(air),(France),1.523776,1.056270
498,(björk),(France),1.473775,1.039770
307,(portishead),(France),1.458055,1.046297
674,"(radiohead, muse)",(France),1.294222,1.026380
112,(pink floyd),(France),1.290107,1.045470
489,(david bowie),(France),1.284477,1.028257
663,(massive attack),(France),1.180330,1.017729
14,(radiohead),(France),1.133150,1.043265


In [18]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Italy'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/3546127947.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Italy'}]


,consequents,antecedents,lift,conviction
933,"(led zeppelin, pink floyd)",(Italy),2.260948,1.063329
439,(bob dylan),(Italy),1.629314,1.053525
59,(pink floyd),(Italy),1.614794,1.101527
196,(led zeppelin),(Italy),1.600376,1.070340
290,(queen),(Italy),1.593870,1.059729
818,"(radiohead, the beatles)",(Italy),1.493684,1.038884
559,(sigur rós),(Italy),1.328225,1.032153
34,(the beatles),(Italy),1.285052,1.063915
432,(depeche mode),(Italy),1.260637,1.028651
609,(franz ferdinand),(Italy),1.201263,1.021799


In [19]:
rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Belgium'}]

C:\Users\vanoy\AppData\Local\Temp/ipykernel_16772/3835654035.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)[rule['antecedents'] == {'Belgium'}]


,consequents,antecedents,lift,conviction
920,(deus),(Belgium),4.823355,1.190488
10102,"(deus, radiohead)",(Belgium),4.577419,1.086837
4375,(hooverphonic),(Belgium),4.223214,1.109031
7284,(eels),(Belgium),3.141016,1.081033
4445,(kings of leon),(Belgium),2.429795,1.084063
4429,(mgmt),(Belgium),2.163110,1.076815
2200,(moby),(Belgium),1.850870,1.081126
6498,(the white stripes),(Belgium),1.773750,1.055296
8563,(madonna),(Belgium),1.576667,1.040639
4315,(bloc party),(Belgium),1.503178,1.047820


При попытке отследить страны, как причину выбора музыкальной группы было выяснено, что некоторые страны имеют предпочтения. К примеру, в Испании и Бельгии предпочитают группу Deus. На Википедии говорится, что это бельгийская группа. Отсюда можно сделать вывод, что группа Deus будет лучше всего собирать концерты в Бельгии


### Правило 5: Что будет, если все положить в анализ и посмотреть?

In [20]:
all_frequent_sets = apriori(user_all, min_support=0.01, use_colnames=True).sort_values(by='support', ascending=False)
rule = association_rules(all_frequent_sets, metric="confidence", min_threshold=0.1)
thing = rule[{'antecedents', 'consequents', 'lift', 'conviction'}].sort_values(by='lift', ascending=False)

In [21]:
thing

,consequents,antecedents,lift,conviction
25359,(iced earth),"(m, helloween)",25.339286,1.960536
25360,"(m, helloween)",(iced earth),25.339286,2.108310
24504,(children of bodom),"(m, arch enemy)",24.635417,2.599013
24505,"(m, arch enemy)",(children of bodom),24.635417,1.685291
19157,(gamma ray),"(m, sonata arctica)",24.544865,1.730863
...,...,...,...,...
24081,(f),(u2),0.545769,0.886508
25931,(Spain),"(pink floyd, radiohead)",0.539298,0.796604
8300,(Spain),"(m, air)",0.539298,0.796604
9151,(f),(metallica),0.519780,0.880788


Предсказуемо, что если все взять и с минимальной обработкой отдать на анализ - данные будут неструктурированные. Однако, удивительно прослеживается рекомендации для "мужчин, слушающих X"